In [ ]:
# IMPORT LIBRARIES
##################

import pandas as pd
import numpy as np
import warnings
import os
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from scipy.stats import skew
from collections import defaultdict
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# DATA LOADING
##############

print("LOADING DATASETS")
print("="*50)

anthro_data = pd.read_csv('anthro_data.csv')
clinical_data = pd.read_csv('clinical_data.csv')
dietary_data = pd.read_csv('dietary_data.csv')
socio_data = pd.read_csv('socio_data.csv')

print(f"1. Anthropometric data: {anthro_data.shape}")
print(f"2. Clinical data: {clinical_data.shape}")
print(f"3. Dietary data: {dietary_data.shape}")
print(f"4. Socioeconomic data: {socio_data.shape}")

# MERGING DATASETS
##################

print("\nMERGING DATASETS")
print("="*50)

# Merge on both hhnum and member_code
merged_data = pd.merge(
    clinical_data,
    anthro_data,
    on=['hhnum', 'member_code'],
    how='inner',
    suffixes=('_clinical', '_anthro')
)

merged_data = pd.merge(
    merged_data,
    dietary_data,
    on=['hhnum', 'member_code'],
    how='inner',
    suffixes=('', '_dietary')
)

merged_data = pd.merge(
    merged_data,
    socio_data,
    on=['hhnum', 'member_code'],
    how='inner',
    suffixes=('', '_socio')
)
print(f"Merged Data: {merged_data.shape}")

# ROW FILTERING
###############

# Dropping rows with no FBS values
print("\nDROPPING ROWS WITH NO FBS VALUES")
print("="*50)

# Check FBS column status
print("Before Cleaning")
print(f"   Total rows: {len(merged_data)}")
print(f"   Missing FBS values: {merged_data['fbs'].isna().sum()}")

# Drop rows with missing FBS values
merged_data = merged_data.dropna(subset=['fbs'])
print("\nAfter Cleaning")
print(f"   Total rows: {len(merged_data)}")
print(f"   Missing FBS values: {merged_data['fbs'].isna().sum()}")

# COLUMN FILTERING
##################

print("\nREMOVING DUPLICATE COLUMNS")
print("="*50)

admin_columns = ['hhnum', 'member_code']

# Identify duplicate columns based on suffixes
duplicate_cols = []
for col in merged_data.columns:
    for suffix in ['_clinical', '_anthro', '_dietary', '_socio']:
        if col.endswith(suffix):
            duplicate_cols.append(col)

# Drop only the columns that actually exist
all_cols_to_drop = admin_columns + duplicate_cols
existing_cols_to_drop = [col for col in all_cols_to_drop if col in merged_data.columns]

# Display columns to be removed (admin/duplicates) in a table
admin_drop_df = pd.DataFrame({'': existing_cols_to_drop})
print(f"Administrative/Duplicate Columns to be removed ({len(existing_cols_to_drop)}):")
print(admin_drop_df)

# Drop them
merged_data = merged_data.drop(columns=existing_cols_to_drop, errors='ignore')

# Analyze missing values before dropping
missing_summary = pd.DataFrame({
    'Column': merged_data.columns,
    'Missing_Count': merged_data.isnull().sum(),
    'Missing_Percentage': (merged_data.isnull().sum() / len(merged_data)) * 100
}).sort_values('Missing_Percentage', ascending=False)

# Display columns with >50% missing
high_missing = missing_summary[missing_summary['Missing_Percentage'] > 50]
print(f"\nColumns with >50% missing values ({len(high_missing)}):")
print(high_missing[['Column', 'Missing_Percentage']])

# Drop these columns
columns_to_drop = high_missing['Column'].tolist()
merged_data = merged_data.drop(columns=columns_to_drop)

total_dropped = len(existing_cols_to_drop) + len(columns_to_drop)
print(f"\nDropped {(total_dropped)} columns")
print(f"Merged Data: {merged_data.shape}")




LOADING DATASETS
1. Anthropometric data: (444400, 18)
2. Clinical data: (351701, 29)
3. Dietary data: (234293, 75)
4. Socioeconomic data: (654425, 20)

MERGING DATASETS
Merged Data: (175899, 136)
